# SOAP example

To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

TODO: Compare against an existing SOAP implementation in Python (e.g. quippy)

In [5]:
%env OMP_NUM_THREADS=1

%matplotlib inline
from matplotlib import pyplot as plt

import os, sys
from ase.io import read
sys.path.insert(0,"../build/")

import sys
import time
import rascal
import json

import ase
#from ase.io import read
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
import sys

from multiprocessing.dummy import Pool as ThreadPool
import queue
from copy import deepcopy,copy

import json

from rascal.neighbourlist import get_neighbourlist
from rascal.lib import RepresentationManager, FeatureManager
from rascal.representations.base import RepresentationFactory,FeatureFactory
from rascal.representations import SphericalExpansion
from rascal.neighbourlist.base import NeighbourListFactory
from rascal.utils import ostream_redirect

env: OMP_NUM_THREADS=1


In [6]:
frames = ase.io.read('../tests/reference_data/dft-smiles_500.xyz',index=':10')
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*1))
print(np.max(list(map(len,new_frames))))

21


In [7]:
test_hypers = {"interaction_cutoff": 6.0, "cutoff_smooth_width": 1.0, "max_radial": 10, "max_angular": 8, "gaussian_sigma_type": "Constant", "gaussian_sigma_constant": 0.5}

In [8]:
sph_expn = rascal.representations.SphericalExpansion(**test_hypers)

In [13]:
rascal.lib._rascal.FeatureManager.BlockSparse_double

rascal.lib._rascal.FeatureManager.BlockSparse_double

In [14]:
with ostream_redirect():
    expansions = sph_expn.transform(new_frames)

In [15]:
X = expansions.get_feature_matrix().T

In [20]:
X.shape

(174, 3240)

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn.transform(new_frames)

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

In [ ]:
expansions.get_feature_matrix()

Good, should be 5 centres (5 atoms) * 10 radial * 81 ${}=(l+1)^2$

In [9]:
import sys
sys.path.insert(0,'../build/')
import json,ubjson
import ase
import argparse
import rascal
import rascal.lib as lrl
import numpy as np
from ase.io import read

def load_json(fn):
    with open(fn,'r') as f:
        data = json.load(f)
    return data[str(data['ids'][0])]
def json2ase(f):
    return ase.Atoms(**{v:f[k] for k,v in
dict(positions='positions',atom_types='numbers',pbc='pbc',cell='cell').items()
})



def get_soap_vectors(hypers, frames):
    with lrl._rascal.utils.ostream_redirect():
        sph_expn = rascal.representations.SphericalExpansion(**hypers)
        expansions = sph_expn.transform(frames)
        soap_vectors = expansions.get_feature_matrix()
    return soap_vectors

##########################################################################################

#dump spherical expansion
def dump_reference_json():
    import ubjson
    import os
    from copy import copy
    path = '../' #should be changed
    sys.path.insert(0, os.path.join(path, 'build/'))
    sys.path.insert(0, os.path.join(path, 'tests/'))

    cutoffs = [2, 3]
    gaussian_sigmas = [0.2, 0.5]
    max_radials = [4, 10]
    max_angulars = [3, 6]

    fns = [
        #os.path.join(path,"tests/reference_data/CaCrP2O7_mvc-11955_symmetrized.json"),
        os.path.join(path,"tests/reference_data/small_molecule.json")
    ]
    fns_to_write = [
        #"reference_data/CaCrP2O7_mvc-11955_symmetrized.json",
         "reference_data/small_molecule.json",

]

    data = dict(filenames=fns_to_write,
                cutoffs=cutoffs,
                gaussian_sigmas=gaussian_sigmas,
                max_radials=max_radials,
                rep_info=[])

    for fn in fns:
        for cutoff in cutoffs:
            data['rep_info'].append([])
            for gaussian_sigma in gaussian_sigmas:
                for max_radial in max_radials:
                    for max_angular in max_angulars:
                        frames = [json2ase(load_json(fn))]
                        hypers = {"interaction_cutoff": cutoff,
                                  "cutoff_smooth_width": 0.0,
                                  "max_radial": max_radial,
                                  "max_angular": max_angular,
                                  "gaussian_sigma_type": "Constant",
                                  "gaussian_sigma_constant": gaussian_sigma}
                        x = get_soap_vectors(hypers, frames)
                        data['rep_info'][-1].append(dict(feature_matrix=x.tolist(),
                                                 hypers=copy(hypers)))
    return data

In [10]:
data = dump_reference_json()

In [12]:
OM = data['rep_info'][0][0]['feature_matrix']
hypers = data['rep_info'][0][0]['hypers']
hypers

{'interaction_cutoff': 2,
 'cutoff_smooth_width': 0.0,
 'max_radial': 4,
 'max_angular': 3,
 'gaussian_sigma_type': 'Constant',
 'gaussian_sigma_constant': 0.2}

In [ ]:
test = np.einsum('nlm,alm->nal',OM,OM)

In [13]:
ref = np.load('PS0_nconf1_sigma0.2_lmax3_nmax4_cutoff2_cweight1.0_ncut-1_natoms.npy')

In [14]:
data['filenames']

['reference_data/CaCrP2O7_mvc-11955_symmetrized.json',
 'reference_data/small_molecule.json']

In [15]:
with open('../tests/reference_data/spherical_expansion_reference.ubjson','wb') as f:
    ubjson.dump(data,f)

In [14]:
with open('../tests/reference_data/spherical_expansion_reference.ubjson','rb') as f:
    data_ref = ubjson.load(f)

In [15]:

for a_r,a_t in zip(data_ref['rep_info'],data['rep_info']):
    for b_r,b_t in zip(a_r,a_t):
        f_r = b_r['feature_matrix']
        f_t = b_t['feature_matrix']

        print(np.allclose(f_r,f_t))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [41]:
8*7**2

392

In [53]:
len(data['rep_info'])

2

In [45]:
np.array(data['rep_info'][0][0]['feature_matrix']).shape

(1568, 11)

In [48]:
data['rep_info'][0][1]['feature_matrix'][2]

[1.2821100555830826e-05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0031758261305962145,
 0.0031758261305962145]

SyntaxError: invalid syntax (get_kernel.py, line 32)